In [4]:
import re
import qiskit
import json
import time

import numpy as np
from typing import List
from qiskit import *
import time

from fast_generator import fc_tree_commute_lookahead_fast
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
from circuit_generator import construct_qcc_circuit

import numpy as np
from rustiq import pauli_network_synthesis, Metric
from rustiq.utils import entangling_count, entangling_depth

In [16]:
#Compare json files in a specific folder
import os, json
def run_experiment_folder(folder_path = None, filename = None, save_output = False, lookahead_size = 5):

    results = []
    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
            end_time = time.time()
            combined_time = end_time - start_time
            
            # Measure time for Qiskit method
            start_time = time.time()
            origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
            origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
            end_time = time.time()
            qiskit_time = end_time - start_time

            # Measure time for RustiQ method
            start_time = time.time()
            circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
            end_time = time.time()
            rustiq_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time,
                    "qiskit_time": qiskit_time,
                    "rustiq_time": rustiq_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0),
                    "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                    "rustiq_method": entangling_count(circuit)
                },
                "test_paulis_file": f'benchmarks/results/test_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results/test_' + filename, 'w') as paulis_file:
                    json.dump(test_paulis, paulis_file, indent=4)
    

    

In [17]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, lookahead_size = 5, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time

    # Measure time for RustiQ method
    start_time = time.time()
    circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
    end_time = time.time()
    rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            "qiskit_time": qiskit_time,
            "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            "rustiq_method": entangling_count(circuit)
        },
        "test_paulis_file": f'benchmarks/results/test_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [18]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10, 12]
orbitals_list = [4, 6, 8, 12, 16, 20, 24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = False, lookahead_size = 5)

{'num_paulis': 24, 'times': {'our_time': 0.15928292274475098, 'combined_time': 0.1647181510925293, 'qiskit_time': 0.14137792587280273, 'rustiq_time': 0.0015349388122558594}, 'gate_counts': {'our_method': 30, 'combined_method': 27, 'qiskit_method': 41, 'rustiq_method': 33}, 'test_paulis_file': 'benchmarks/results/test_Paulis'}
{'num_paulis': 80, 'times': {'our_time': 0.6555271148681641, 'combined_time': 1.1727478504180908, 'qiskit_time': 0.5271539688110352, 'rustiq_time': 0.010827064514160156}, 'gate_counts': {'our_method': 115, 'combined_method': 99, 'qiskit_method': 181, 'rustiq_method': 162}, 'test_paulis_file': 'benchmarks/results/test_Paulis'}
{'num_paulis': 320, 'times': {'our_time': 2.8113720417022705, 'combined_time': 3.0446360111236572, 'qiskit_time': 2.3107829093933105, 'rustiq_time': 0.11585521697998047}, 'gate_counts': {'our_method': 514, 'combined_method': 466, 'qiskit_method': 1003, 'rustiq_method': 794}, 'test_paulis_file': 'benchmarks/results/test_Paulis'}


KeyboardInterrupt: 

In [19]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = False, lookahead_size = 10)

LiH.json
{'num_paulis': 61, 'times': {'our_time': 0.694976806640625, 'combined_time': 0.6314878463745117, 'qiskit_time': 0.551727294921875, 'rustiq_time': 0.006796121597290039}, 'gate_counts': {'our_method': 81, 'combined_method': 80, 'qiskit_method': 180, 'rustiq_method': 114}, 'test_paulis_file': 'benchmarks/results/test_LiH.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 2.42276930809021, 'combined_time': 2.680933952331543, 'qiskit_time': 1.2490220069885254, 'rustiq_time': 0.03375101089477539}, 'gate_counts': {'our_method': 317, 'combined_method': 311, 'qiskit_method': 786, 'rustiq_method': 351}, 'test_paulis_file': 'benchmarks/results/test_H2O.json'}
benzene.json
{'num_paulis': 1254, 'times': {'our_time': 23.081979274749756, 'combined_time': 27.582254886627197, 'qiskit_time': 11.417005062103271, 'rustiq_time': 5.437843322753906}, 'gate_counts': {'our_method': 3320, 'combined_method': 3306, 'qiskit_method': 7602, 'rustiq_method': 3355}, 'test_paulis_file': 'benchmarks/resu

In [25]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = False, lookahead_size = 10)

max_cut_benchmark_n14_e60_l5.json
{'num_paulis': 370, 'times': {'our_time': 17.992774963378906, 'combined_time': 18.126566171646118, 'qiskit_time': 0.47733306884765625, 'rustiq_time': 0.08726882934570312}, 'gate_counts': {'our_method': 428, 'combined_method': 428, 'qiskit_method': 588, 'rustiq_method': 463}, 'test_paulis_file': 'benchmarks/results/test_max_cut_benchmark_n14_e60_l5.json'}
max_cut_benchmark_n10_e45_l5.json
{'num_paulis': 275, 'times': {'our_time': 9.232853174209595, 'combined_time': 8.723498821258545, 'qiskit_time': 0.3853471279144287, 'rustiq_time': 0.0377810001373291}, 'gate_counts': {'our_method': 274, 'combined_method': 274, 'qiskit_method': 448, 'rustiq_method': 290}, 'test_paulis_file': 'benchmarks/results/test_max_cut_benchmark_n10_e45_l5.json'}
